## Objection detection 
#### This should be run on a smaller sample of images selected as a result of my previous script
This ipython notebook is running in Tensor flow environment and not Anocanda!

In [2]:
!pip install matplotlib

    100% |████████████████████████████████| 12.8MB 59kB/s 
    100% |████████████████████████████████| 61kB 3.9MB/s 
    100% |████████████████████████████████| 194kB 1.4MB/s 
    100% |████████████████████████████████| 491kB 1.3MB/s 
    100% |████████████████████████████████| 61kB 3.7MB/s 
  Running setup.py bdist_wheel for subprocess32 ... - \ | / done
  Stored in directory: /Users/Mehdi/Library/Caches/pip/wheels/7d/4c/a4/ce9ceb463dae01f4b95e670abd9afc8d65a45f38012f8030cc
Successfully built subprocess32


In [9]:
!pip install opencv-python

    100% |████████████████████████████████| 27.2MB 28kB/s 
    100% |████████████████████████████████| 4.4MB 174kB/s 
  Found existing installation: numpy 1.11.2
    Uninstalling numpy-1.11.2:
      Successfully uninstalled numpy-1.11.2


In [11]:
!pip install exifread

  Using cached ExifRead-2.1.2-py2-none-any.whl


In [27]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import numpy as np
import exifread
import glob, os
import shutil
import tensorflow as tf
import pickle
import sklearn as sk
import re

### Downloading the pre-trained Inception v3 model and puting it in model_dir/imagenet/

In [25]:
model_dir='/Users/Mehdi/Downloads/GoldenGate-Animals/Inception3/imagenet/'

In [13]:
!python /Users/Mehdi/anaconda/envs/tensorflow/lib/python2.7/site-packages/tensorflow/models/image/imagenet/classify_image.py --model_dir /Users/Mehdi/Downloads/GoldenGate-Animals/Inception3/imagenet

>> Downloading inception-2015-12-05.tgz 100.0%
Succesfully downloaded inception-2015-12-05.tgz 88931400 bytes.
W tensorflow/core/framework/op_def_util.cc:332] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca (score = 0.89233)
indri, indris, Indri indri, Indri brevicaudatus (score = 0.00859)
lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens (score = 0.00264)
custard apple (score = 0.00141)
earthstar (score = 0.00107)


In [17]:
# Making a list for images
#list_images = [images_dir+f for f in os.listdir(images_dir) if re.search('jpg|JPG', f)]
images_dir='/Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-13_20160315/'
imlist=glob.glob(images_dir+"*.JPG")

In [19]:
# Making the graph definition and structure for Tensor flow
# This function is using classify_image_graph_def.pb file which is downloaded as part of the Inception v3 model
def graphmaker():
    with tf.python.platform.gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')


In [23]:
def extract_features(imlist):
    nb_features = 2048
    features = np.empty((len(imlist),nb_features))
    labels = []

    graphmaker()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

        for ind, image in enumerate(imlist):
            if (ind%100 == 0):
                print('Processing %s...' % (image))
            if not tf.python.platform.gfile.Exists(image):
                tf.logging.fatal('File does not exist %s', image)

            image_data = tf.python.platform.gfile.FastGFile(image, 'rb').read()
            predictions = sess.run(next_to_last_tensor,
            {'DecodeJpeg/contents:0': image_data})
            features[ind,:] = np.squeeze(predictions)
            labels.append(re.split('_\d+',image.split('/')[1])[0])

    return features, labels

In [28]:
# This might take a while
features,labels = extract_features(imlist)


Processing /Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-13_20160315/SM1 2-10-2016 11-52-022.JPG...
Processing /Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-13_20160315/SM1 2-14-2016 2-01-46 PM.JPG...
Processing /Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-13_20160315/SM1 2-21-2016 8-53-46 AM.JPG...
Processing /Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-13_20160315/SM1 2-28-2016 12-37-030.JPG...
Processing /Users/Mehdi/Downloads/GoldenGate-Animals/SM-1_SB-13_20160315/SM1 3-2-2016 11-53-025.JPG...


In [ ]:
# Saving the feature vectors
pickle.dump(features, open('features', 'wb'))
pickle.dump(labels, open('labels', 'wb'))


### Classification

In [ ]:
# Loading the features
features = pickle.load(open('features'))
labels = pickle.load(open('labels'))

In [ ]:
# Making train/test sets
X_train, X_test, y_train, y_test = sk.cross_validation.train_test_split(features, labels, test_size=0.2, random_state=42)


In [ ]:
# Classifying with Linear SVM
clf = sk.svm.LinearSVC(C=1.0, loss='squared_hinge', penalty='l2',multi_class='ovr')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [ ]:
# Plotting the confussion metrix
def plot_confusion_matrix(y_true,y_pred):
    cm_array = sk.metrics.confusion_matrix(y_true,y_pred)
    true_labels = np.unique(y_true)
    pred_labels = np.unique(y_pred)
    plt.imshow(cm_array[:-1,:-1], interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix", fontsize=16)
    cbar = plt.colorbar(fraction=0.046, pad=0.04)
    cbar.set_label('Number of images', rotation=270, labelpad=30, fontsize=12)
    xtick_marks = np.arange(len(true_labels))
    ytick_marks = np.arange(len(pred_labels))
    plt.xticks(xtick_marks, true_labels, rotation=90)
    plt.yticks(ytick_marks,pred_labels)
    plt.tight_layout()
    plt.ylabel('True label', fontsize=14)
    plt.xlabel('Predicted label', fontsize=14)
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 12
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size


In [ ]:
print("Accuracy: {0:0.1f}%".format(sk.metrics.accuracy_score(y_test,y_pred)*100))
plot_confusion_matrix(y_test,y_pred)